In [ ]:
"""
!pip install gymnasium==1.1.1
!pip install gymnasium-robotics==1.3.1
!pip install numpy==1.25.2
!pip install torch==2.7.0
"""

In [ ]:
import gymnasium as gym
from gymnasium.spaces import Box,Dict
from gymnasium.vector import AsyncVectorEnv,SyncVectorEnv
import gymnasium_robotics
gym.register_envs(gymnasium_robotics)

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
from dataclasses import dataclass

In [ ]:
@dataclass
class Hypers:
    num_env : int = 1

hypers = Hypers()

In [ ]:
class FetchReachCustom(gym.Wrapper):
    def __init__(self,env : gym.Env):
        super().__init__(env)
        self.action_space = Box(-1,1,(3,),np.float32)
        self.observation_space = Dict(
            {
            "observation" : Box(-np.inf,np.inf,(3,),np.float64),
            "achieved_goal" : Box(-np.inf,np.inf,(3,),np.float64),
            "desired_goal" : Box(-np.inf,np.inf,(3,),np.float64)
            }
        )
    
    def process_obs(self,observation):
        observation["observation"] = observation["observation"][:3]
        return observation
         
    def step(self, action):
        observation, reward, done,truncated, info = self.env.step(action)
        return  self.process_obs(observation), reward, done,truncated, info
    
    def reset(self,seed=None, options=None):
        observation,info = self.env.reset(seed=seed,options=options)
        return self.process_obs(observation),info


def tranform_observation(observation_dict : gym.spaces.Dict): # -> torch.Size([6])
    observation = observation_dict.get("observation")
    target = observation_dict.get("achieved_goal")
    assert observation.shape == target.shape
    output = np.concatenate((observation,target),axis=-1)
    return torch.from_numpy(output).to(torch.float32)

def sync_env():
    def make_env():
        x = gym.make("FetchReach-v3")
        x = FetchReachCustom(x)
        return x
    return SyncVectorEnv([make_env for _ in range(hypers.num_env)])

In [ ]:
class Actor(nn.Module):
    def __init__(self):
        super().__init__()
        pass


class Critic(nn.Module):
    def __init__(self):
        super().__init__()
        pass
    